<a href="https://colab.research.google.com/github/guanlin-chen08/laravel-test/blob/main/finetune_segmentation(with_test)_last.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!pip install -U segmentation-models-pytorch albumentations --user

     |████████████████████████████████| 71kB 5.1MB/s 
     |████████████████████████████████| 81kB 6.7MB/s 
     |████████████████████████████████| 245kB 12.3MB/s 
     |████████████████████████████████| 61kB 7.7MB/s 
     |████████████████████████████████| 952kB 9.3MB/s 
     |████████████████████████████████| 36.7MB 88kB/s 
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-cp36-none-any.whl size=60963 sha256=26f292cb72b88c3ee745e5367380fff29a95b4641eb8df2abba40ead49c4b019
  Stored in directory: /root/.cache/pip/wheels/69/df/63/62583c096289713f22db605aa2334de5b591d59861a02c2ecd
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.6.3-cp36-none-any.whl size=12421 sha256=4f674c3b2cbfb96e67e2a8b0ec30c4c3b80da05c2f62acf307c3c79feea10fdb
  Stored in directory: /root/.cache/pip/wheels/42/1e/a9/2a578ba9ad04e776e80bf0f70d8a7f4c29ec0718b92d8f6ccd
Successfully built pretrainedmodels efficientnet-pytorch


In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

import numpy as np
import cv2
import matplotlib.pyplot as plt

# **Dataloader**

In [ ]:
from torch.utils.data import DataLoader
from torch.utils.data import Dataset as BaseDataset

#**Augmentations**

In [ ]:
import albumentations as albu

In [ ]:
import torch
import numpy as np
import segmentation_models_pytorch as smp

ModuleNotFoundError: ignored

In [ ]:
import tensorflow as tf
from torch.utils.tensorboard import SummaryWriter
import numpy as np
logdir = './logs'
writer = SummaryWriter('./logs')

In [ ]:
import os
import torch
import segmentation_models_pytorch as smp
import numpy as np
import cv2
import matplotlib.pyplot as plt
import albumentations as albu
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset as BaseDataset

os.environ['CUDA_VISIBLE_DEVICES'] = '0'
DATA_DIR = './gdrive/MyDrive/Colab Notebooks/SYNTHIA4/'


x_train_dir = os.path.join(DATA_DIR, 'Train/RGB')
y_train_dir = os.path.join(DATA_DIR, 'Train/SemSeg')

x_valid_dir = os.path.join(DATA_DIR, 'Validation/RGB')
y_valid_dir = os.path.join(DATA_DIR, 'Validation/SemSeg')


# helper function for data visualization
def visualize(**images):
    """PLot images in one row."""
    n = len(images)
    plt.figure(figsize=(16, 5))
    for i, (name, image) in enumerate(images.items()):
        plt.subplot(1, n, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.title(' '.join(name.split('_')).title())
        plt.imshow(image)
    plt.show()

def get_training_augmentation():
    train_transform = [

        albu.HorizontalFlip(p=0.5),

        albu.ShiftScaleRotate(scale_limit=0.5, rotate_limit=0, shift_limit=0.1, p=1, border_mode=0),

        albu.PadIfNeeded(min_height=320, min_width=320, always_apply=True, border_mode=0),
        albu.RandomCrop(height=320, width=320, always_apply=True),

        albu.IAAAdditiveGaussianNoise(p=0.2),
        albu.IAAPerspective(p=0.5),

        albu.OneOf(
            [
                albu.CLAHE(p=1),
                albu.RandomBrightness(p=1),
                albu.RandomGamma(p=1),
            ],
            p=0.9,
        ),

        albu.OneOf(
            [
                albu.IAASharpen(p=1),
                albu.Blur(blur_limit=3, p=1),
                albu.MotionBlur(blur_limit=3, p=1),
            ],
            p=0.9,
        ),

        albu.OneOf(
            [
                albu.RandomContrast(p=1),
                albu.HueSaturationValue(p=1),
            ],
            p=0.9,
        ),
    ]
    return albu.Compose(train_transform)


def get_validation_augmentation():
    """Add paddings to make image shape divisible by 32"""
    test_transform = [
        albu.PadIfNeeded(480, 640)
    ]
    return albu.Compose(test_transform)


def to_tensor(x, **kwargs):
    return x.transpose(2, 0, 1).astype('float32')


def get_preprocessing(preprocessing_fn):
    """Construct preprocessing transform
    
    Args:
        preprocessing_fn (callbale): data normalization function 
            (can be specific for each pretrained neural network)
    Return:
        transform: albumentations.Compose
    
    """
    
    _transform = [
        albu.Lambda(image=preprocessing_fn),
        albu.Lambda(image=to_tensor, mask=to_tensor),
    ]
    return albu.Compose(_transform)

class Dataset(BaseDataset):
    """CamVid Dataset. Read images, apply augmentation and preprocessing transformations.
    
    Args:
        images_dir (str): path to images folder
        masks_dir (str): path to segmentation masks folder
        class_values (list): values of classes to extract from segmentation mask
        augmentation (albumentations.Compose): data transfromation pipeline 
            (e.g. flip, scale, etc.)
        preprocessing (albumentations.Compose): data preprocessing 
            (e.g. noralization, shape manipulation, etc.)
    
    """
    
    CLASSES = [ 'misc', 'sky', 'building', 'road', 'sidewalk', 
                'fence', 'vegetation', 'pole', 'car', 'sign', 
                'pedestrian', 'cyclist', 'lane-marking']

    
    def __init__(
            self, 
            images_dir, 
            masks_dir, 
            classes=None, 
            augmentation=None, 
            preprocessing=None,
    ):
        self.ids = os.listdir(images_dir)
        self.images_fps = [os.path.join(images_dir, image_id) for image_id in self.ids]
        self.masks_fps = [os.path.join(masks_dir, image_id) for image_id in self.ids]
        
        # convert str names to class values on masks
        self.class_values = [self.CLASSES.index(cls.lower()) for cls in classes]
        
        self.augmentation = augmentation
        self.preprocessing = preprocessing
    
    def __getitem__(self, i):
        
        # read data
        image = cv2.imread(self.images_fps[i])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        mask = cv2.imread(self.masks_fps[i], 0)
        
        # extract certain classes from mask (e.g. cars)
        masks = [(mask == v) for v in self.class_values]
        mask = np.stack(masks, axis=-1).astype('float')
        
        # apply augmentations
        if self.augmentation:
            sample = self.augmentation(image=image, mask=mask)
            image, mask = sample['image'], sample['mask']
        
        # apply preprocessing
        if self.preprocessing:
            sample = self.preprocessing(image=image, mask=mask)
            image, mask = sample['image'], sample['mask']
            
        return image, mask
        
    def __len__(self):
        return len(self.ids)


ENCODER = 'se_resnext50_32x4d'
ENCODER_WEIGHTS = 'imagenet'
CLASSES = [ 'misc', 'sky', 'building', 'road', 'sidewalk', 
            'fence', 'vegetation', 'pole', 'car', 'sign', 
            'pedestrian', 'cyclist', 'lane-marking']
ACTIVATION = 'sigmoid' # could be None for logits or 'softmax2d' for multicalss segmentation
DEVICE = 'cuda'

# create segmentation model with pretrained encoder
model = smp.FPN(
    encoder_name=ENCODER, 
    encoder_weights=ENCODER_WEIGHTS, 
    classes=len(CLASSES), 
    activation=ACTIVATION,
)

preprocessing_fn = smp.encoders.get_preprocessing_fn(ENCODER, ENCODER_WEIGHTS)


train_dataset = Dataset(
    x_train_dir, 
    y_train_dir, 
    augmentation=get_training_augmentation(), 
    preprocessing=get_preprocessing(preprocessing_fn),
    classes=CLASSES,
)

valid_dataset = Dataset(
    x_valid_dir, 
    y_valid_dir, 
    augmentation=get_validation_augmentation(), 
    preprocessing=get_preprocessing(preprocessing_fn),
    classes=CLASSES,
)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=12)
valid_loader = DataLoader(valid_dataset, batch_size=2, shuffle=False, num_workers=4)

loss = smp.utils.losses.DiceLoss()
metrics = [
    smp.utils.metrics.IoU(threshold=0.5),
]

optimizer = torch.optim.Adam([ 
    dict(params=model.parameters(), lr=0.0001),
])

# create epoch runners 
# it is a simple loop of iterating over dataloader`s samples
train_epoch = smp.utils.train.TrainEpoch(
    model, 
    loss=loss, 
    metrics=metrics, 
    optimizer=optimizer,
    device=DEVICE,
    verbose=True,
)

valid_epoch = smp.utils.train.ValidEpoch(
    model, 
    loss=loss, 
    metrics=metrics, 
    device=DEVICE,
    verbose=True,
)


max_score = 0

for i in range(0, 40):
    
    print('\nEpoch: {}'.format(i))
    train_logs = train_epoch.run(train_loader)
    valid_logs = valid_epoch.run(valid_loader)
    
    # do something (save model, change lr, etc.)
    if max_score < valid_logs['iou_score']:
        max_score = valid_logs['iou_score']
        torch.save(model, './best_model.pth')
        print('Model saved!')
        
    if i == 25:
        optimizer.param_groups[0]['lr'] = 1e-5
        print('Decrease decoder learning rate to 1e-5!')

    writer.add_scalar('Loss/train', train_logs['dice_loss'], i)
    writer.add_scalar('Loss/valid', valid_logs['dice_loss'], i)
    writer.add_scalar('Accuracy/train', train_logs['iou_score'], i)
    writer.add_scalar('Accuracy/valid', valid_logs['iou_score'], i)

In [ ]:
! kill -9 828

In [ ]:
del /q %TMP%.tensorboard-info*

In [ ]:
%reload_ext tensorboard

In [ ]:
%tensorboard --logdir logs

In [ ]:
import os
import torch
import segmentation_models_pytorch as smp
import numpy as np
import cv2
import matplotlib.pyplot as plt
import albumentations as albu
from torch.utils.data import DataLoader
from torch.utils.data import Dataset as BaseDataset

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

DATA_DIR = './gdrive/MyDrive/Colab Notebooks/SYNTHIA4/'

x_test_dir = os.path.join(DATA_DIR, 'Test/RGB')
y_test_dir = os.path.join(DATA_DIR, 'Test/SemSeg')

best_model = torch.load('./best_model.pth')

# create test dataset
test_dataset = Dataset(
    x_test_dir, 
    y_test_dir, 
    augmentation=get_validation_augmentation(), 
    preprocessing=get_preprocessing(preprocessing_fn),
    classes=CLASSES,
)

test_dataloader = DataLoader(test_dataset)

# evaluate model on test set
test_epoch = smp.utils.train.ValidEpoch(
    model=best_model,
    loss=loss,
    metrics=metrics,
    device=DEVICE,
)

logs = test_epoch.run(test_dataloader)

In [ ]:
test_dataset_vis = Dataset(
    x_test_dir, y_test_dir, 
    classes=CLASSES,
)

color=[[255,0,0],[0,255,0],[0,0,255],[255,255,0],[255,0,255],
        [0,255,255],[67,125,255],[125,200,67],[180,67,125],[67,40,0],[100,67,20],
        [10,140,67],[67,210,19],[110,120,67]]

for i in range(30):
    
    gt_img = np.zeros([480,640,3],dtype=np.uint8)
    gt_img.fill(0) # or img[:] = 255

    output_img = np.zeros([480,640,3],dtype=np.uint8)
    output_img.fill(0) # or img[:] = 255

    image_vis = test_dataset_vis[i][0].astype('uint8')
    image, gt_mask = test_dataset[i]

    gt_mask = gt_mask.squeeze()

    for j in range(len(gt_mask)):
        for x in range(480):
            for y in range(640):
                if gt_mask[j][x][y]==1:
                    gt_img[x][y]=color[j]
        
    x_tensor = torch.from_numpy(image).to(DEVICE).unsqueeze(0)
    pr_mask = best_model.predict(x_tensor)
    pr_mask = (pr_mask.squeeze().cpu().numpy().round())
    
    for j in range(len(pr_mask)):
        for x in range(480):
            for y in range(640):
                if pr_mask[j][x][y]==1:
                    output_img[x][y]=color[j]
    if i == 1:
      visualize(
        ground_truth_mask=gt_img, 
        predicted_mask=output_img
        )
      

In [ ]:
visualize(
    ground_truth_mask=gt_img, 
    predicted_mask=output_img
)